<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data/train/mountain/art1131.jpg?raw=1)|![](https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data/validation/forest/cdmc317.jpg?raw=1)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to add additional full connected layers to our model. When we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

In [1]:
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

In [2]:
for layer in resnet.layers:
    layer.trainable = False

In [3]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [4]:
import tensorflow as tf
import os

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

In [5]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [6]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training cat pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training dog pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation cat pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation dog pictures

In [7]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [8]:
print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 254
total training forest images: 270
total validation mountain images: 125
total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [9]:
batch_size = 16
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [11]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [12]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


## Instatiate Model

In [37]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 222, 222, 224)     6272      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 111, 111, 224)     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 109, 109, 64)      129088    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 186624)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                11944000  
_________________________________________________________________
dense_11 (Dense)             (None, 2)                

## Fit Model

In [38]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 3s 98ms/step - loss: 0.1810 - accuracy: 0.9621 - val_loss: 0.3541 - val_accuracy: 0.9034
Epoch 2/50
32/32 [==============================] - 3s 93ms/step - loss: 0.0169 - accuracy: 0.9940 - val_loss: 0.6724 - val_accuracy: 0.9091
Epoch 3/50
32/32 [==============================] - 3s 93ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.7968 - val_accuracy: 0.9091
Epoch 4/50
32/32 [==============================] - 3s 95ms/step - loss: 3.4430e-04 - accuracy: 1.0000 - val_loss: 0.7095 - val_accuracy: 0.9148
Epoch 5/50
32/32 [==============================] - 3s 93ms/step - loss: 1.3167e-04 - accuracy: 1.0000 - val_loss: 0.6936 - val_accuracy: 0.9205
Epoch 6/50
32/32 [==============================] - 3s 94ms/step - loss: 7.4313e-05 - accuracy: 1.0000 - val_loss: 0.7211 - val_accuracy: 0.9148
Epoch 7/50
32/32 [==============================] - 3s 93ms/step - loss: 6.6174e-05 - accuracy: 1.0000 - val_loss: 0.5147 - val_accuracy: 0.94

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [15]:
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from IPython.display import display
from PIL import Image

In [34]:
# Define the Model
model = Sequential([
    Conv2D(224, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

In [35]:
# Compile Model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 222, 222, 224)     6272      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 111, 111, 224)     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 109, 109, 64)      129088    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 186624)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                11944000  
_________________________________________________________________
dense_11 (Dense)             (None, 2)                

In [36]:
history = model.fit_generator(train_data_gen,
                              epochs=epochs,
                              validation_data=val_data_gen)

Epoch 1/50
34/34 [==============================] - 3s 99ms/step - loss: 0.8549 - accuracy: 0.6567 - val_loss: 0.1948 - val_accuracy: 0.9179
Epoch 2/50
34/34 [==============================] - 3s 93ms/step - loss: 0.3907 - accuracy: 0.8330 - val_loss: 0.3321 - val_accuracy: 0.8821
Epoch 3/50
34/34 [==============================] - 3s 93ms/step - loss: 0.3221 - accuracy: 0.8818 - val_loss: 0.3362 - val_accuracy: 0.8462
Epoch 4/50
34/34 [==============================] - 3s 94ms/step - loss: 0.2636 - accuracy: 0.9099 - val_loss: 0.1990 - val_accuracy: 0.9179
Epoch 5/50
34/34 [==============================] - 3s 94ms/step - loss: 0.1622 - accuracy: 0.9362 - val_loss: 0.4054 - val_accuracy: 0.8359
Epoch 6/50
34/34 [==============================] - 3s 93ms/step - loss: 0.1595 - accuracy: 0.9362 - val_loss: 0.1560 - val_accuracy: 0.9436
Epoch 7/50
34/34 [==============================] - 3s 93ms/step - loss: 0.1167 - accuracy: 0.9475 - val_loss: 0.4286 - val_accuracy: 0.8205
Epoch 8/50
34

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

In [62]:
from tensorflow.keras.applications.vgg19 import preprocess_input

train_datagen = ImageDataGenerator(
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    preprocessing_function=preprocess_input,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    data_format='channels_last')

train_datagen.flow_from_directory(batch_size=batch_size,
                                  directory=train_dir,
                                  shuffle=True,
                                  target_size=(IMG_HEIGHT, IMG_WIDTH),
                                  class_mode='binary')

valid_datagen.flow_from_directory(batch_size=batch_size,
                                  directory=validation_dir,
                                  shuffle=True,
                                  target_size=(IMG_HEIGHT, IMG_WIDTH),
                                  class_mode='binary')

Found 533 images belonging to 2 classes.
Found 195 images belonging to 2 classes.


In [65]:
model = Model(resnet.input, predictions)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "functional_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
______________________________________________________________________________________

In [66]:
history = model.fit_generator(train_data_gen,
                              epochs=epochs,
                              validation_data=val_data_gen)

Epoch 1/50
34/34 [==============================] - 9s 250ms/step - loss: nan - accuracy: 0.5159 - val_loss: nan - val_accuracy: 0.3436
Epoch 2/50
34/34 [==============================] - 8s 226ms/step - loss: nan - accuracy: 0.5159 - val_loss: nan - val_accuracy: 0.3436
Epoch 3/50
34/34 [==============================] - 8s 224ms/step - loss: nan - accuracy: 0.5159 - val_loss: nan - val_accuracy: 0.3436
Epoch 4/50
34/34 [==============================] - 8s 224ms/step - loss: nan - accuracy: 0.5159 - val_loss: nan - val_accuracy: 0.3436
Epoch 5/50
34/34 [==============================] - 8s 224ms/step - loss: nan - accuracy: 0.5159 - val_loss: nan - val_accuracy: 0.3436
Epoch 6/50
34/34 [==============================] - 8s 223ms/step - loss: nan - accuracy: 0.5159 - val_loss: nan - val_accuracy: 0.3436
Epoch 7/50
34/34 [==============================] - 8s 225ms/step - loss: nan - accuracy: 0.5159 - val_loss: nan - val_accuracy: 0.3436
Epoch 8/50
34/34 [==============================

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language